In [1]:
from pathlib import Path
import onnxruntime as ort
import cv2
import imageio
import numpy as np
from tqdm import tqdm
from speed_trapv3.tracking.tracking import get_video_properties, transform_image#, get_frame_box
from speed_trapv3.tracking.config import Config
from sparrow_datums import AugmentedBoxTracking, BoxTracking, FrameBoxes, PType

In [ ]:
video_path = "/code/data/datasets/common_hall/source_video/25_resampled_vid.mp4"
model_path = "/code/data/models/detection/model.onnx"
temp_save_path = "/code/data/datasets/temp_imgs"

In [2]:
np.array([Config.image_width, Config.image_height] * 2)

array([1280,  720, 1280,  720])

In [4]:
def get_frame_box(
    boxes: np.ndarray,
    scores: np.ndarray,
    labels: np.ndarray,
    image_width: int,
    image_height: int,
) -> tuple[FrameBoxes, FrameBoxes]:
    """Convert bounding boxes from numpy array into ball and player FrameBoxes.
    Parameters
    ----------
    boxes
        (n, 4) boxes in absolute tlbr format with shape Config.image_size
    scores
        Confidence that each box is an object
    labels
        Class indices (0 == ball, 1 == active player)
    Returns
    -------
    ball_boxes, player_boxes
        Output boxes with width/height corresponding to the input video
    """
    relative_boxes = boxes / np.array([Config.image_width, Config.image_height] * 2)
    vehicle_mask = np.logical_and(
        labels == Config.vehicle_class_index, scores > Config.vehicle_score_threshold
    )
    vehicle_boxes = FrameBoxes(
        relative_boxes[vehicle_mask],
        PType.relative_tlbr,
        image_width=image_width,
        image_height=image_height,
    )
    return vehicle_boxes

In [5]:
video_path = Path(video_path)
slug = video_path.name.removesuffix(".mp4")
sess = ort.InferenceSession(str(model_path), providers=["CUDAExecutionProvider"])
fps, n_frames = get_video_properties(video_path)
reader = imageio.get_reader(video_path)
for i in tqdm(range(n_frames)):
    # data = reader.get_data(i)
    data = cv2.imread("/code/data/datasets/images/WBuBqS9h8_261.jpg")
    data = cv2.rectangle(
        data, (450, 200), (1280, 720), thickness=5, color=(0, 255, 0)
    )
    input_height, input_width = data.shape[:2]
    x = transform_image(data)
    boxes, scores, labels = sess.run(None, {"input": x[0]})
    vehicle_boxes = get_frame_box(
        boxes, scores, labels, image_width=input_width, image_height=input_height
    )
    vehicle_boxes = vehicle_boxes.to_absolute()
    vehicle_boxes = vehicle_boxes.to_tlbr()
    cv2.imwrite(temp_save_path + f"/{i}.jpg", data)

2022-10-25 21:43:20.363665676 [W:onnxruntime:, session_state.cc:1030 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2022-10-25 21:43:20.363679243 [W:onnxruntime:, session_state.cc:1032 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.
 23%|██▎       | 140/599 [00:12<00:39, 11.51it/s]


KeyboardInterrupt: 

In [6]:
vehicle_boxes

FrameBoxes([[ 277.68785265,    0.        ,  389.69523112,    0.        ],
            [   0.        ,  307.87399292,  224.91179742,  307.87399292],
            [2275.55555556,    0.        , 2275.55555556,    0.        ],
            [1859.18424479,   55.68155193, 1859.18424479,   55.68155193],
            [2275.55555556,    0.        , 2275.55555556,    0.        ],
            [   0.        ,    0.        ,  195.58865017,    0.        ]])

In [7]:
boxes

array([[ 1.56199417e+02,  0.00000000e+00,  2.19203568e+02,
         0.00000000e+00],
       [ 0.00000000e+00,  5.47331543e+02,  1.26512886e+02,
         5.47331543e+02],
       [ 1.28000000e+03,  0.00000000e+00,  1.28000000e+03,
         0.00000000e+00],
       [ 1.04579114e+03,  9.89894257e+01,  1.04579114e+03,
         9.89894257e+01],
       [ 1.28000000e+03,  0.00000000e+00,  1.28000000e+03,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.10018616e+02,
         0.00000000e+00],
       [ 1.28000000e+03,  0.00000000e+00,  1.28000000e+03,
         0.00000000e+00],
       [ 9.03964600e+02,  0.00000000e+00,  9.03973145e+02,
         0.00000000e+00],
       [ 8.76972656e+02,  0.00000000e+00,  1.05073535e+03,
         0.00000000e+00],
       [ 8.00010193e+02,  7.20000000e+02,  8.00010193e+02,
         7.20000000e+02],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00],
       [-1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
      

: 

In [ ]:
img_h, img_w, _ = data.shape
pop = (boxes / np.array([img_w, img_h, img_w, img_h]))

In [ ]:
boxes.astype(int)